In [ ]:
import os
from glob import glob
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

import pydicom
from pymedphys.type import DeliveryData
from pymedphys.dcm import anonymise_dicom
from pymedphys.coll import find_relevant_control_points
from pymedphys.logfile import delivery_data_from_logfile
from pymedphys.gamma import gamma_dcm, convert_to_percent_pass

In [ ]:
data_root = r'C:\Users\sbiggs\logfile2dicom'

In [ ]:
logfile_dose_dicom_path = glob(os.path.join(
    data_root, 'calced_dose', '*Logfile*.dcm'
))[0]

logfile_dose_dicom_path

In [ ]:
patient_dose_dicom_path = glob(os.path.join(
    data_root, 'calced_dose', '*BREAST*.dcm'
))[0]

patient_dose_dicom_path

In [ ]:
gamma_dcm?

In [ ]:
def calc_and_plot(dose_threshold, distance_threshold, random_subset=None):
    gamma = gamma_dcm(
        patient_dose_dicom_path, logfile_dose_dicom_path, 
        dose_threshold, distance_threshold, local_gamma=True, random_subset=random_subset)
    
    valid_gamma = gamma[~np.isnan(gamma)]

    num_points = len(valid_gamma)
    weights = np.ones(num_points)/num_points

    plt.hist(valid_gamma, 20, weights=weights)
    plt.title('Gamma histogram 2% / {} mm | 20% lower dose cutoff'.format(distance_threshold))
    plt.xlabel('Local Gamma')
    plt.ylabel('Normalised bin count')

    max_x_axis = np.ceil(np.max(valid_gamma))

    plt.xlim([0, max_x_axis])
    plt.show()
    
    return gamma

In [ ]:
gamma = []

for i in range(3):
    gamma.append(calc_and_plot(2, i + 1))
    
save_filepath = os.path.join(
    data_root, 'gamma_calcs_2percent.npz'
)

gamma_dict = {
    '1': gamma[0],
    '2': gamma[1],
    '3': gamma[2]
}

np.savez_compressed(save_filepath, **gamma_dict)

In [ ]:
gamma = []

for i in range(3):
    gamma.append(calc_and_plot(1, i + 1))
    
save_filepath = os.path.join(
    data_root, 'gamma_calcs_1percent.npz'
)

gamma_dict = {
    '1': gamma[0],
    '2': gamma[1],
    '3': gamma[2]
}

np.savez_compressed(save_filepath, **gamma_dict)